# Assigment 2 - Python dictionaries and file I/O

### Overview
------------

The GTEx gene expression dataset consists of samples taken from many human subjects across a variety of tissues. From this dataset we have identified a set of highly-expressed tissue-specific genes. In this exercise, you will be pulling the expressing data for those genes but only in their primary tissue of expression and investigating how much variability exists across the sample population.

Biological Learning Objectives

- Use metadata to associate data with subject or sample traits
- Use descriptive statistics to make inferences about gene expression heterogeneity

Computational Learning Objectives

- Create and modify a dicitionary
- Use keys to retrieve values stored in a dictionary
- Step through each key and value in a dictionary inside a `for` loop
- Open and read a text file
- Write to a text file

### Instructions
----------------

- Save a copy of this notebook as `~/qbXX-answers/python_dictionaries.ipynb`.
- Fill in answers in the available code/markdown cells below.
- Remember to comment your code to help yourself and us know what each part is intended to do.

In [8]:
# Use this dictionary for identifying genes of interest and their corresponding tissue of interest

gene_tissue = {
    'ENSG00000042832.11': 'Thyroid',
    'ENSG00000091704.9': 'Pancreas',
    'ENSG00000118245.2': 'Testis',
    'ENSG00000122304.10': 'Testis',
    'ENSG00000122852.14': 'Lung',
    'ENSG00000132693.12': 'Liver',
    'ENSG00000134812.7': 'Stomach',
    'ENSG00000135346.8': 'Pituitary',
    'ENSG00000137392.9': 'Pancreas',
    'ENSG00000142515.14': 'Prostate',
    'ENSG00000142615.7': 'Pancreas',
    'ENSG00000142789.19': 'Pancreas',
    'ENSG00000158874.11': 'Liver',
    'ENSG00000162438.11': 'Pancreas',
    'ENSG00000164816.7': 'Small Intestine - Terminal Ileum',
    'ENSG00000164822.4': 'Small Intestine - Terminal Ileum',
    'ENSG00000168925.10': 'Pancreas',
    'ENSG00000168928.12': 'Pancreas',
    'ENSG00000170890.13': 'Pancreas',
    'ENSG00000171195.10': 'Minor Salivary Gland',
    'ENSG00000172179.11': 'Pituitary',
    'ENSG00000175535.6': 'Pancreas',
    'ENSG00000175646.3': 'Testis',
    'ENSG00000182333.14': 'Stomach',
    'ENSG00000187021.14': 'Pancreas',
    'ENSG00000203784.2': 'Testis',
    'ENSG00000204983.13': 'Pancreas',
    'ENSG00000219073.7': 'Pancreas',
    'ENSG00000229859.9': 'Stomach',
    'ENSG00000240338.5': 'Pancreas',
    'ENSG00000254647.6': 'Pancreas',
    'ENSG00000256713.7': 'Stomach',
    'ENSG00000259384.6': 'Pituitary',
    }

1. Load sample metadata from "~/Data/GTEx/GTEx_Analysis_v8_Annotations_SampleAttributesDS.txt" and create a dictionary of sample metadata associating each sample ID with the tissue it was sampled from

    - This file is tab-separated but some values have spaces in their text so make sure to specify `\t` in the `.split()` method
    - There are several columns in the metadata file, but you only need to be concerned with `SAMPID` and `SMTSD`, the sample ID and sample tissue, respectively
    - Your dictionary should use the `SAMPID` index [0] as the key and `SMTSD` as the value

In [9]:
metadata_fname = "/Users/cmdb/Data/GTEx/GTEx_Analysis_v8_Annotations_SampleAttributesDS.txt"
md_file = open(metadata_fname)
meta_dictionary = {} # Create a dictionary to hold the metadata
header= md_file.readline() # the readline reads until the first newline character, use this an print it such that you can find the index positions for the columns you are interested in
for line in md_file: # Step through each line of the metadata file, stepping through a line converts to a list
    if line == header:
        continue
    line = line.strip('\n')
    line = line.split('\t') # Remove the newline character from the line you just read in and split it by the tab character
    meta_dictionary[line[0]] = line[6]  # the dictionary has to be in the forloop because you are adding the key for each line, refer to the fruit slide
    # Add the data to your metadata dictionary, using the "SAMPID" column value as the dictionary key and the "SMTSD" column value as the dictionary value

2. Load and process the headeer from the GTEx gene expression file "~/Data/GTEx/GTEx_Analysis_2017-06-05_v8_RNASeQCv1.1.9_gene_tpm.gct.gz"

    - You have code that will open the file and read in the header line
    - Create a list of tissues corresponding to the column names in the header, one tissue per column name (don't worry about which the column is a sample ID or not since column names that aren't sample IDs will just get a "missing" label)
    - For each value in the header, see if it appears in the dictionary of sample ID/tissues that you created in step 1
    - If the value is in your dictionary, add the corresponding tissue name (the value in dictionary for that sample ID key) to your tissue list
    - If the value is not in your dictionary, add "missing" to the tissue list

In [10]:
# Because this file is a gzip-compressed file, you will be using the `gzip` library that comes with python
import gzip

expression_fname = "/Users/cmdb/Data/GTEx/GTEx_Analysis_2017-06-05_v8_RNASeQCv1.1.9_gene_tpm.gct.gz"

# Open the file using the gzip library
fs = gzip.open(expression_fname)

# The file starts with two lines of information before the header, so these are skipped using 2 calls of `.readline()` without keeping the returned data
_ = fs.readline()
_ = fs.readline()

# Gzipped text files are read in as byte strings, not regular strings, so you will see that `.decode()` is included after `.readline()` to convert the input into a string
header = fs.readline().decode().rstrip().split("\t")

# Make sure to close the file when you are done reading it
fs.close()

# Create a list to hold the tissue names
tissues = []

# Step through each value in the header
for value in header:
    if value not in meta_dictionary:
        tissues.append('Missing')
    else:
        tissues.append(meta_dictionary.get(value)) # you want to print the value onto this because it will take the individual index position and stick it onto the list. appending header would take that whole long list and append it every time
# Check if the value has an entry in the gene_tissue dictionary, adding either the tissue corresponding to that entry or "missing" to your tissues list
# Example of the start of the print(tissues) statement. ['Missing', 'Missing', 'Adipose - Subcutaneous', 'Muscle - Skeletal', 'Artery - Tibial'...


3. Create a list of expression values for each gene in the `gene_tissue` dictionary (given at the start of the exercise), keeping only expression values from the tissue of interest for that gene

    - Since the file is already open, you can keep reading lines from it in, picking up from after the last line read
    - You will be log-transforming the expression data using the `log2` function from the built-in library `math`
    - To deal with zeros in expression data, add one before log-transforming expression data

In [11]:
# Import the log2 function from the math library to transform the expression counts
from math import log2

# Create a dictionary to hold each gene's expression data
gene_expression = {}

# Because this is a large file, for testing purposes you can use only a small portion of the file to test your code
line_counter = 0

# Step through each line of the file
for each in gzip.open(expression_fname):

    # Convert the line into a string from a byte string, remove the newline character at the end, and split it by tab characters
    fields = each.decode().rstrip().split("\t")

    # Check if this line contains data for a gene we are interested in. If not, skip it
    fields = each.decode().rstrip().split("\t")
    if fields[0] not in gene_tissue:
        continue

    # Determine which tissue we care about for this gene
    tissue = gene_tissue[fields[0]]

    # Create a list in the gene expression dictionary to hold the gene's data
    gene_expression[fields[0]] = []

    # Look at each column position, one by one
    for i in range(len(tissues)):

        # For each column, see if it is from the tissue we care about (using the tissues list). If so, add that field to our gene expression list, first transforming it with log2(1 + float(field[i])) 
        if tissues[i] == tissue:
            gene_expression[fields[0]].append(log2(1 + float(fields[i])))


4. For each gene, calculate the mean, standard deviation, and relative standard deviation and write them to a results file

    - You will be using the `mean` and `stdev` functions from the built-in `statistics` library. Both functions accept a list as their input
    - Format your results for each gene into a string and write that string to your results file
    - If you want to organize your results better (completely optional!!), you can save your results in a list and sorting them by tissue type before writing them to a file

In [12]:
# Load the functions to caculcate the mean and standard deviation from a list
from statistics import stdev, mean

# Open a file to write the results to
output = open('day2hw1.txt', 'w')

# Step through each gene and its expression value list
sorting = []
for gene in gene_expression:
    expression_list = gene_expression[gene] #access entries in adictionary using square brackets #access values by using name of dictionary, then the value you want to match to the key
    #expression_list = gene_expression.get(gene)  # Calculate the mean and standard deviation of expression
    gene_mean = mean(expression_list)
    gene_stdev = stdev(expression_list)
    gene_reldev= gene_stdev / gene_mean
    tis = gene_tissue[gene] #so the point of this is to match the gene name to to tissue type. in part 4m you have the gene_expression in a dictionary so when you parse thru this dictionary, the index variable (gene) is defined by all of the gene expression key is taking on the values of interest from the gene_expression index. You can find the matching tissue using the gene variable which is the key from the gene_tissue dictionary
    sorting.append([tis,gene_reldev]) #remember that you can append this as a list, by containing your variables as a bracket, this command is grouping two variables as a list, such that they are in a long list of grouped sublists
    print (f'The tisue name is {tis}, the gene name is {gene}, the mean is  {gene_mean}, the stdev is {gene_stdev}, the relative stdev is {gene_reldev}')
    output.write(f'The tisue name is {tis}, the gene name is {gene}, the mean is  {gene_mean}, the stdev is {gene_stdev}, the relative stdev is {gene_reldev}\n')
    # Calculate the relative standard deviation (coefficient of variation, std / mean)
    
    # Write the tissue namne, gene name, mean, standard deviation, and relative standard deviation to your results file

# Close your results file
output.close()
sorting.sort() #sorting my list of just tissue name and relative std dev
for one in sorting: #printing each sublist as an individual line to make the analysis much easier
    print(one)



The tisue name is Pancreas, the gene name is ENSG00000162438.11, the mean is  11.64813927955005, the stdev is 0.757491204101109, the relative stdev is 0.06503109088255767
The tisue name is Pancreas, the gene name is ENSG00000142615.7, the mean is  11.982260919355417, the stdev is 1.4113620914457623, the relative stdev is 0.1177876279731093
The tisue name is Pancreas, the gene name is ENSG00000219073.7, the mean is  13.186023575042132, the stdev is 0.6691398656550582, the relative stdev is 0.05074614510181628
The tisue name is Pancreas, the gene name is ENSG00000142789.19, the mean is  14.651792140427176, the stdev is 0.5222110990262671, the relative stdev is 0.03564144877440514
The tisue name is Testis, the gene name is ENSG00000203784.2, the mean is  10.588661238478307, the stdev is 1.2485963275387792, the relative stdev is 0.11791824286544222
The tisue name is Liver, the gene name is ENSG00000132693.12, the mean is  11.233820059075176, the stdev is 3.2303438267149205, the relative st

5. Looking at your results, answer the following questions

    - Which tissues have low relative variability (rv < 0.2), medium relative variability (0.2 < rv < 0.4), or high relative variability (rv > 0.4)?
    - Do each of the genes for a given tissue show the same level of relative variability?
    - Propose an explanation for differences in relative variability you see between tissues based on what you know about their functions.
    - Were any of the results surpising to you?

The tissues with low relative variability (rv<0.2) are the pancreas, pituitary, testis, and thyroid. The liver, lung, small intestine, minor salivary gland, prostate have medium relative variability (0.2< rv < 0.4). The stomach had a high relative variability (rv > 0.4). However, some of the tissue types- for example the liver- had differing relative variability depending on what genes were expressed, thus falling into the medium variability classification.

It really depends on the tissue type. For example, the four genes in the testis tissue had a relative variability ranging between 0.1 and 0.11. This is very consistent juxtaposed to the spread of relative variability seen in the Liver; Where one gene has rv~0.05 and the other has rv~0.2.

The differences in relative variability seen between the tissues could be a result of the gene-function and cell-type variability seen in the different tissue types. For example, one gene may be reserved for a specific function in a rare cell type, and therefore has a higher variability compared to a housekeeping gene which might be necessary for the identity of all cell types in the tissue.


I was surprised by the high relative varability of genes seen in the Stomach. I would have assumed that with such a specialized function, there would be relatively stable expression of genes within this tissue. However, upon further reflection, I understand that the variety of substrates the stomach encounters must require specialized gene arrays for the most effective digestion. Depending on what substrates are present at the time, could be an effector for what genes get expressed, making it a highly malleable system.

Answers:
